In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
columns = ["Country", "Product Category", "Product_Name", "Illustrative Price", "Shipped Quantity", "Average Days Late", "Order Entry Date", "Requested Delivery Date", "Latest Actual Delivery Date", "Latest Actual Delivery Date Fiscal Quarter Year"]
data = pd.read_csv("USAID_GHSC-PSM_Health_Commodity_Delivery_Dataset_20240105.csv", header= 0)
# is order entry different than requested delivery date

data = data[columns]
data
# data.groupby(["Product Category", "Latest Actual Delivery Date Fiscal Quarter Year"]).

C:\Users\quang\AppData\Local\Temp\ipykernel_9064\3028048542.py:2: DtypeWarning: Columns (74,75,76,88) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("USAID_GHSC-PSM_Health_Commodity_Delivery_Dataset_20240105.csv", header= 0)


,Country,Product Category,Product_Name,Illustrative Price,Shipped Quantity,Average Days Late,Latest Actual Delivery Date,Latest Actual Delivery Date Fiscal Quarter Year
0,Mozambique,Others,"Extended Warranty, GeneXpert GXIV-4, 1 Year, 1...",2898.00,1,-4.0,"Wednesday, December 27, 2017",2018-Q1
1,Mozambique,Others,"Extended Warranty, GeneXpert GXIV-4, 1 Year, 1...",2898.00,1,-2.0,"Friday, December 29, 2017",2018-Q1
2,Mozambique,Others,"Extended Warranty, GeneXpert GXIV-4, 1 Year, 1...",2898.00,1,-4.0,"Wednesday, December 27, 2017",2018-Q1
3,Mozambique,Others,"Extended Warranty, GeneXpert GXIV-4, 1 Year, 1...",2898.00,1,-4.0,"Wednesday, December 27, 2017",2018-Q1
4,Mozambique,Others,"Extended Warranty, GeneXpert GXIV-4, 1 Year, 1...",2898.00,1,-3.0,"Thursday, December 28, 2017",2018-Q1
...,...,...,...,...,...,...,...,...
38500,Zimbabwe,Essential Medicines,"Lidocaine 2% (20 mL) Vial, 5 Vials",5.93,3892,1.0,"Friday, April 13, 2018",2018-Q3
38501,Senegal,Long Lasting Insecticide Treated Nets (LLINs),"(Campaign Bulk) LLIN 190x180x170 cm (LxWxH), P...",2.79,486022,0.0,"Tuesday, April 5, 2022",2022-Q3
38502,Zimbabwe,Voluntary Male Circumcision (VMMC) Kits,"MC Kit, Sterile, Single Use, For All Surgical ...",12.48,49759,18.0,"Friday, April 20, 2018",2018-Q3
38503,Tanzania,Voluntary Male Circumcision (VMMC) Kits,"MC Kit, Sterile, Single Use, For All Surgical ...",12.48,16500,70.0,"Wednesday, January 24, 2018",2018-Q2


In [ ]:
# Wednesday, December 27, 2017	-> YYYYMMDD


In [10]:
def reformat_date(line):
    datetime.strptime(line, "%A, %B %d, %Y").strftime("%d/%m/%Y")
    return line

data["Latest Actual Delivery Date"] = data["Latest Actual Delivery Date"].apply(reformat_date)
table = pd.pivot_table(data, values= ["Average Days Late"],
                       index= ["Latest Actual Delivery Date Fiscal Quarter Year", "Product Category", "Product_Name", "Illustrative Price", "Shipped Quantity"],
                       columns= "Latest Actual Delivery Date")
table



Average Days Late  \
Latest Actual Delivery Date                                                                                                                                                        Friday, April 1, 2022   
Latest Actual Delivery Date Fiscal Quarter Year Product Category                            Product_Name                                       Illustrative Price Shipped Quantity                         
2016-Q3                                         Male Condoms                                Male Condom (Latex) Lubricated, No Logo, 53 mm,... 96.87              1728                               NaN   
2016-Q4                                         Contraceptive Implants                      Etonogestrel 68 mg/Rod, 1 Rod Implant, 1 Each      8.50               9288                               NaN   
                                                                                                                                                                  10008                              NaN   
                                                                                            Levonorgestrel 75 mg/Rod w/ Trocar, 5 Year Effi... 8.50               38000                              NaN   
                                                                                                                                                                  145000                             NaN   
...                                                                                                                                                                                                  ...   
2023-Q4                                         Voluntary Male Circumcision (VMMC) Supplies MC, Locking Ring Device, Shang Ring Device with... 2785.00            40                                 NaN   
                                                                                            MC, Locking Ring Device, Shang Ring Device with... 2785.00            45                                 NaN   
                                                                                            MC, Locking Ring Device, Shang Ring Device with... 2785.00            45                                 NaN   
                                                                                            MC, Locking Ring Device, Shang Ring Device with... 2785.00            40                                 NaN   
                                                                                            MC, Locking Ring Device, Shang Ring Device with... 2785.00            20                                 NaN   

                                                                                                                                                                                                           \
Latest Actual Delivery Date                                                                                                                                                        Friday, April 10, 2020   
Latest Actual Delivery Date Fiscal Quarter Year Product Category                            Product_Name                                       Illustrative Price Shipped Quantity                          
2016-Q3                                         Male Condoms                                Male Condom (Latex) Lubricated, No Logo, 53 mm,... 96.87              1728                                NaN   
2016-Q4                                         Contraceptive Implants                      Etonogestrel 68 mg/Rod, 1 Rod Implant, 1 Each      8.50               9288                                NaN   
                                                                                                                                                                  10008                               NaN   
                                                                                            Levo

In [ ]:
# One column - Average Days Late (Y)
# X: product categories "Product_Name" "Illustrative Price", "Shipped Quantity"
# index: Actual Date